In [1]:
import numpy as np
import pandas as pd
import wisps
import wisps.simulations as wispsim

Adding 89 sources from /Users/caganze/research/splat//resources/Spectra/Public/MAGE/ to spectral database
Adding 2404 sources from /Users/caganze/research/splat//resources/Spectra/Public/SPEX-PRISM/ to spectral database
Adding 145 sources from /Users/caganze/research/splat//resources/Spectra/Public/LRIS-RED/ to spectral database












In [21]:
import numpy as np 
from astropy.coordinates import SkyCoord
from multiprocessing import Pool
from scipy.interpolate import interp1d
import scipy.integrate as integrate
import bisect
import numba

#contants
PNTS=[]
DISTANCE_LIMITS=[]
SPGRID=n.array([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41])
Rsun=8300.
Zsun=27.
HS=[150,200,250, 300,350,400, 450, 500]
INTERPOLATED_CDFS=[]
#redefine magnitude limits by taking into account the scatter for each pointing 
#use these to compute volumes

#REDEFINED_MAG_LIMITS={'F110':    23.054573, 'F140':    23.822972, 'F160' :   23.367867}

#-------------------------------------------
def random_draw(xvals, cdfvals, nsample=10):
    """
    randomly drawing from a discrete distribution
    """
    @numba.vectorize("int32(float64)")
    def invert_cdf(i):
        return bisect.bisect(cdfvals, i)-1
    x=np.random.rand(nsample)
    idx=invert_cdf(x)
    return np.array(xvals)[idx]

def density_function(r, z, h=300.):
    
    """
    A custom juric density function that only uses numpy arrays for speed
    All units are in pc
    """
    l = 2600. # radial length scale of exponential thin disk 
    zpart=np.exp(-abs(z-Zsun)/h)
    rpart=np.exp(-(r-Rsun)/l)
    return zpart*rpart


def custom_volume(l,b,dmin, dmax, h):
    nsamp=1000
    ds = np.linspace(dmin,dmax,nsamp)
    rd=np.sqrt( (ds * np.cos( b ) )**2 + Rsun * (Rsun - 2 * ds * np.cos( b ) * np.cos( l ) ) )
    zd=Zsun+ ds * np.sin( b - np.arctan( Zsun / Rsun) )
    rh0=density_function(rd, zd,h=h )
    val=integrate.trapz(rh0*(ds**2), x=ds)
    return val

def interpolated_cdf(pntname, h):
    pnt= POINTINGS[pntname]
    l, b= pnt.coord.galactic.l.radian, pnt.coord.galactic.b.radian
    d=np.concatenate([[0], np.logspace(-1, np.log10(5000), int(1e2))])
    cdfvals=np.array([custom_volume(l,b,0, dx, h) for dx in d])
    return interp1d(d, cdfvals)

def draw_distance_with_cdf(pntname, dmin, dmax, nsample, h):
    #draw distances using inversion of the cumulative distribution 
    d=np.logspace(np.log10(dmin), np.log10(dmax), int(nsample))
    cdfvals=(INTERPOLATED_CDFS[h][pntname])(d)
    return wisps.random_draw(d, cdfvals/np.nanmax(cdfvals), int(nsample))


In [2]:
pnts=pd.read_pickle(wisps.OUTPUT_FILES+'/pointings_correctedf110.pkl')

In [17]:
def lb(pnt):
    l, b= pnt.coord.galactic.l.radian, pnt.coord.galactic.b.radian
    return pd.Series({'l':l, 'b': b}, index=[pnt.name])

In [22]:
wispsim.SPGRID


array([17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41])